In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
users = pd.read_csv('../data/' + 'users.csv')
books = pd.read_csv('../data/' + 'books.csv')
train = pd.read_csv('../data/' + 'train_ratings.csv')
test = pd.read_csv('../data/' + 'test_ratings.csv')
sub = pd.read_csv('../data/' + 'sample_submission.csv')

In [3]:
#user의 location 전처리

In [4]:
users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '', regex=True) # 특수문자 제거

users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
users = users.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.


In [ ]:
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass
    

In [6]:
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]
    

In [ ]:
users.isna().sum()

In [ ]:
modify_location = users[(users['location_state'].isna())&(users['location_city'].notnull())]['location_city'].values

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_state'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass
    

In [9]:
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]
    

In [10]:
users.isna().sum()

user_id                 0
location                0
age                 27833
location_city         122
location_state        606
location_country      266
dtype: int64

In [ ]:
#books의 출판사 전처리

In [16]:
books['img_path']

0         images/0002005018.01.THUMBZZZ.jpg
1         images/0060973129.01.THUMBZZZ.jpg
2         images/0374157065.01.THUMBZZZ.jpg
3         images/0399135782.01.THUMBZZZ.jpg
4         images/0425176428.01.THUMBZZZ.jpg
                        ...                
149565    images/067161746X.01.THUMBZZZ.jpg
149566    images/0767907566.01.THUMBZZZ.jpg
149567    images/0884159221.01.THUMBZZZ.jpg
149568    images/0912333022.01.THUMBZZZ.jpg
149569    images/1569661057.01.THUMBZZZ.jpg
Name: img_path, Length: 149570, dtype: object

In [17]:
books[books['isbn']!=books['img_path'].apply(lambda x: x.split('/')[1][:10])]


,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path


135436

In [10]:
books.isna().sum() #language, category, summary에 결측값이 있습니다

isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language               67227
category               68851
summary                67227
img_path                   0
dtype: int64